In [2]:
import numpy as np
import pandas as pd

Código gerado pelo chatGPt apenas para ideação de como fazer a extração dos dados

In [ ]:
import os
import numpy as np
import nibabel as nib
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# Define paths
data_dir = 'path/to/your/data/raw_data/training dataset'
image_dir = 'images'
contour_dir = 'contours'

# Collect file paths
image_paths = []
contour_paths = []

for case_folder in os.listdir(data_dir):
    case_folder_path = os.path.join(data_dir, case_folder)
    
    if os.path.isdir(case_folder_path):
        image_path = os.path.join(case_folder_path, image_dir, f'Case num {case_folder}.nii.gz')
        contour_path = os.path.join(case_folder_path, contour_dir)
        
        if os.path.exists(image_path) and os.path.exists(contour_path):
            image_paths.append(image_path)
            contour_paths.append(contour_path)

# Split data into training and validation sets
train_image_paths, val_image_paths, train_contour_paths, val_contour_paths = train_test_split(
    image_paths, contour_paths, test_size=0.2, random_state=42
)

# Define a function to load and preprocess data
def load_and_preprocess_data(image_paths, contour_paths):
    images = []
    contours = []

    for img_path, cnt_path in zip(image_paths, contour_paths):
        # Load image
        img = nib.load(img_path).get_fdata()
        
        # Load contour data (you might need to adapt this part based on your actual contour data format)
        # Example: Load contour as binary mask
        cnt = np.zeros_like(img)
        for cnt_file in os.listdir(cnt_path):
            cnt_file_path = os.path.join(cnt_path, cnt_file)
            cnt_slice = nib.load(cnt_file_path).get_fdata()
            cnt += cnt_slice

        images.append(img)
        contours.append(cnt)

    images = np.array(images)
    contours = np.array(contours)

    # Normalize images and convert contours to binary (if needed)
    images /= np.max(images)
    contours = (contours > 0).astype(np.uint8)

    return images, contours

# Load training and validation data
train_images, train_contours = load_and_preprocess_data(train_image_paths, train_contour_paths)
val_images, val_contours = load_and_preprocess_data(val_image_paths, val_contour_paths)

# Optional: Convert contours to categorical (if you have multiple classes)
train_contours_categorical = to_categorical(train_contours)
val_contours_categorical = to_categorical(val_contours)
